### Dissertação de Mestrado: Frederico Gomes

• **Autor:** Maria Luiza Campos

• **Data:** Julho 2025

**Metodologia:** Teste Exato de Fisher.

In [1]:
## Importando bibliotecas

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Importando os dados

dados = pd.read_csv('dados_limpos.csv')
print("Dados importados com sucesso!")

Dados importados com sucesso!


# Entendendo o modelo de análise

As variáveis de idade, da forma como estão agrupadas, não comportam um teste exato de Fisher, que necessita de duas categorias não ordinais. 
Mesmo que agrupassemos as idades, ainda não seria ideal porque idade possui ordem cronológica (é uma variável quantitativa ordinal). Assim, nossas categorias possuem
uma ordem de compreensão, sendo 0 indivíduos mais novos e 3 indivíduos mais velhos (em ambos réu e vítima).

As demais variáveis são qualitativas nominais, ou seja, as categorias não possuem uma ordem pré-estabelecida e são todas binárias (0,1). Assim, são variáveis ideais para o teste exato de Fisher. Vamos começar nossa análise com essas demais variáveis.

### Teste exato de Fisher

Esse teste vai nos retornar a existência ou não de uma associação entre as variáveis. Ou seja, não retorna uma correlação - que possui direção e força. O p-valor vai apenas nos informar se existe ou não uma associação válida entre as duas variáveis qualitativas nominais binárias. 

H0: não existe associação entre as variáveis. 

Se p-valor for pequeno, então rejeitamos a hipótese nula e podemos afirmar que há associação entre as variáveis. 

O odds ratio do teste nos fornece informação sobre força e direção da relação. 

### Notas relevantes sobre a amostra 

O número de observações se altera a depender da variável independente analisada. Veja abaixo um modelo simples e genérico de equação que representa, visualmente, uma das associações que estamos investigando:

Resultado da Condenação [f(x) = y] = B0 + B1 * Existência de filhos (x)

Aqui, estamos apenas supondo que o resultado da condenação é uma função da existência de filhos, isto é, a existência de filhos (x, variável independente) afeta o resultado da condenação (y, variável dependente). 

As variáveis independentes na amostra tem um número variável de observações, ou seja, o tamanho da amostra muda a depender de x. Nesse caso, eu vou trabalhar com sub-amostras pra cada x numa tentativa de exaurir completamente a informação disponível e tirar vantagem de possíveis amostras maiores. 




In [2]:
### Analisando a categoria de vínculo

## Filtrando apenas para as variáveis de interesse

variaveis_interesse = ['ID_processo','resultado_julgamento', 'categoria_vinculo']
dados_filtrados = dados[variaveis_interesse].copy()
print(dados_filtrados)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações filtradas: {len(dados_filtrados)}")

     ID_processo  resultado_julgamento  categoria_vinculo
0       148088.0                     1                  1
1       203354.0                     1                  1
2       211280.0                     1                  1
3       264516.0                     1                  1
4       849912.0                     1                  1
..           ...                   ...                ...
132     191972.0                     0                  1
133     240959.0                     0                  1
134     220480.0                     0                  1
135     221039.0                     0                  1
136     220755.0                     0                  1

[137 rows x 3 columns]
Total de observações originais: 137
Total de observações filtradas: 137


In [3]:
## Verificando missing values

for var in variaveis_interesse:
    missing_count = dados_filtrados[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados[var].unique()}")

## Não há missing values, o total de observações com informação completa é 137. 


ID_processo:
  Missing values: 1 (0.7%)
  Valores únicos: [148088. 203354. 211280. 264516. 849912. 804741. 254352. 202422. 224010.
 200221. 184262. 132092. 835236. 173028. 191800. 193481. 694900. 184789.
 174559. 143912. 280782. 215458. 885416. 913497. 208333. 212840. 871265.
 249385. 157443. 239068. 259005. 176898. 176979. 527774. 171531. 176559.
 169452. 149635. 175618. 140372. 143672. 155742. 158776. 231453. 134780.
 195164. 831541. 800685. 795482. 210952. 208602. 722014. 681080. 676155.
 197973. 197199. 208631. 237957. 147619. 183064. 201216. 169551. 208489.
 226153. 185259. 630372. 188501. 189990. 142704. 163831. 153688. 379763.
 477088. 382196. 387667. 160739. 185437. 163595. 176254. 171010. 136005.
 165091. 155347. 158511. 162216. 154015. 728575. 110419. 283470. 966382.
 170792. 216534. 961744. 159217. 270098. 211850. 214764. 231698. 248059.
 279235. 172188. 118368. 221858.     nan 220261. 976227. 280596. 101478.
 293118. 283931. 230572. 201531. 265254. 240573. 238961. 201966. 2

In [5]:
## Vamos rodar o Teste Exato de Fisher para cada uma das variáveis de interesse

## Definindo uma função para o teste

def teste_fisher(var1, var2, dados):
    """
    Executa o teste exato de Fisher entre duas variáveis de interesse

    """
    # Criar tabela de contigê ncia
    tabela = pd.crosstab(dados[var1], dados[var2])

    print(f"\n=== Teste de Fisher: {var1} vs {var2} ===")
    print("Tabela de contigência")
    print(tabela)

    # verificar se a tabela é válida para Fisher
    if tabela.shape != (2, 2):
        print(f"⚠️  Tabela {tabela.shape} - Fisher pode não ser apropriado")
        return None, None, None

    # Executar teste de fisher
    try:
        odds_ratio, p_value = stats.fisher_exact(tabela)

        print(f"Odds Ratio: {odds_ratio:.4f}")
        print(f"P-value: {p_value:.6f}")

         # Interpretação
        if p_value < 0.001:
            sig = "*** (p < 0.001)"
        elif p_value < 0.01:
            sig = "** (p < 0.01)"
        elif p_value < 0.05:
            sig = "* (p < 0.05)"
        else:
            sig = "ns (p >= 0.05)"
        
        print(f"Significância: {sig}")
        
        return odds_ratio, p_value, sig
        
    except Exception as e:
        print(f"❌ Erro no teste: {e}")
        return None, None, None


In [6]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 1: resultado_julgamento vs categoria_vinculo

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'categoria_vinculo', dados_filtrados)
resultados['resultado_julgamento_vs_categoria_vinculo'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs categoria_vinculo ===
Tabela de contigência
categoria_vinculo      0   1
resultado_julgamento        
0                      0  24
1                     36  77
Odds Ratio: 0.0000
P-value: 0.000527
Significância: *** (p < 0.001)

resultado_julgamento_vs_categoria_vinculo:
  Odds Ratio: 0.0000
  P-valor: 0.000527
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Categoria de Vínculo**

• Odds Ratio: 0.

• P-valor: 0.000410

1. Odds ratio = 0: Uma das células da tabela de contingência é zero. Nenhum caso do tipo abusivo na amostra foi absolvido. 

2.  Dos casos com vínculo afetivo na amostra, 77 foram condenados e 24 absolvidos. 

3.  Dos casos considerados como abuso na amostra, 36 foram condenados. 

**Conclusão:** *Ter vínculo afetivo pode alterar a decisão do julgamento*.

**• O p-valor está abaixo de 0.05 (0.000527), indicando uma associação forte entre resultado do julgamento e tipo de vínculo.**

In [7]:
## Analisando Coabitação

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_2 = ['ID_processo','resultado_julgamento', 'coab']
dados_filtrados_2 = dados[variaveis_interesse_2].copy()
print(dados_filtrados_2)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_2)}")

     ID_processo  resultado_julgamento  coab
0       148088.0                     1   0.0
1       203354.0                     1   0.0
2       211280.0                     1   0.0
3       264516.0                     1   0.0
4       849912.0                     1   0.0
..           ...                   ...   ...
132     191972.0                     0   1.0
133     240959.0                     0   1.0
134     220480.0                     0   1.0
135     221039.0                     0   0.0
136     220755.0                     0   1.0

[137 rows x 3 columns]
Total de observações originais: 137
Total de observações final: 137


In [8]:
## Verificando missing values

for var in variaveis_interesse_2:
    missing_count = dados_filtrados_2[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_2)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_2[var].unique()}")

## um dos missing values é o ID do processo. Vamos garantir que se elimine as observações com missing values apenas
## nas variáveis de interesse. 

ID_processo:
  Missing values: 1 (0.7%)
  Valores únicos: [148088. 203354. 211280. 264516. 849912. 804741. 254352. 202422. 224010.
 200221. 184262. 132092. 835236. 173028. 191800. 193481. 694900. 184789.
 174559. 143912. 280782. 215458. 885416. 913497. 208333. 212840. 871265.
 249385. 157443. 239068. 259005. 176898. 176979. 527774. 171531. 176559.
 169452. 149635. 175618. 140372. 143672. 155742. 158776. 231453. 134780.
 195164. 831541. 800685. 795482. 210952. 208602. 722014. 681080. 676155.
 197973. 197199. 208631. 237957. 147619. 183064. 201216. 169551. 208489.
 226153. 185259. 630372. 188501. 189990. 142704. 163831. 153688. 379763.
 477088. 382196. 387667. 160739. 185437. 163595. 176254. 171010. 136005.
 165091. 155347. 158511. 162216. 154015. 728575. 110419. 283470. 966382.
 170792. 216534. 961744. 159217. 270098. 211850. 214764. 231698. 248059.
 279235. 172188. 118368. 221858.     nan 220261. 976227. 280596. 101478.
 293118. 283931. 230572. 201531. 265254. 240573. 238961. 201966. 2

In [9]:
## Vamos remover esses valores faltantes da tabela filtrada

nas_de_interesse = ['resultado_julgamento', 'coab']
dados_limpos_2 = dados_filtrados_2.dropna(subset = nas_de_interesse)

print(len(dados_filtrados_2), '→', len(dados_limpos_2), 'observações após dropna nessas colunas')

## Isso significa que temos 135 observações com informação COMPLETA. 

137 → 135 observações após dropna nessas colunas


In [10]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 2: resultado_julgamento vs coabitação

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'coab', dados_limpos_2)
resultados['resultado_julgamento_vs_coabitacao'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs coab ===
Tabela de contigência
coab                  0.0  1.0
resultado_julgamento          
0                       6   17
1                      83   29
Odds Ratio: 0.1233
P-value: 0.000022
Significância: *** (p < 0.001)

resultado_julgamento_vs_coabitacao:
  Odds Ratio: 0.1233
  P-valor: 0.000022
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Coabitação**

• Odds Ratio: 0.1233.

• P-valor: 0.000022

1. Odds ratio = 0.1233. A chance de ser condenado quando há coabitação é menor do que quando não há coabitação.

2.  Dos casos com coabitação, 29 foram condenados e 17 absolvidos. 

3.  Dos casos sem coabitação, 83 foram condenados e 6 absolvidos. 

**Conclusão:** *a coabitação altera a decisão do julgamento*.

**• O p-valor está abaixo de 0.05 (0.000022), indicando uma associação forte entre resultado do julgamento e coabitação.**


In [11]:
## Analisando existência de filhos

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_3 = ['ID_processo','resultado_julgamento', 'filhos']
dados_filtrados_3 = dados[variaveis_interesse_3].copy()
print(dados_filtrados_3)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_3)}")

     ID_processo  resultado_julgamento  filhos
0       148088.0                     1     0.0
1       203354.0                     1     0.0
2       211280.0                     1     0.0
3       264516.0                     1     0.0
4       849912.0                     1     1.0
..           ...                   ...     ...
132     191972.0                     0     1.0
133     240959.0                     0     0.0
134     220480.0                     0     1.0
135     221039.0                     0     0.0
136     220755.0                     0     1.0

[137 rows x 3 columns]
Total de observações originais: 137
Total de observações final: 137


In [12]:
## Verificando missing values

for var in variaveis_interesse_3:
    missing_count = dados_filtrados_3[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_3)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_3[var].unique()}")

## um dos missing values é o ID do processo. Vamos garantir que se elimine as observações com missing values apenas
## nas variáveis de interesse. 

ID_processo:
  Missing values: 1 (0.7%)
  Valores únicos: [148088. 203354. 211280. 264516. 849912. 804741. 254352. 202422. 224010.
 200221. 184262. 132092. 835236. 173028. 191800. 193481. 694900. 184789.
 174559. 143912. 280782. 215458. 885416. 913497. 208333. 212840. 871265.
 249385. 157443. 239068. 259005. 176898. 176979. 527774. 171531. 176559.
 169452. 149635. 175618. 140372. 143672. 155742. 158776. 231453. 134780.
 195164. 831541. 800685. 795482. 210952. 208602. 722014. 681080. 676155.
 197973. 197199. 208631. 237957. 147619. 183064. 201216. 169551. 208489.
 226153. 185259. 630372. 188501. 189990. 142704. 163831. 153688. 379763.
 477088. 382196. 387667. 160739. 185437. 163595. 176254. 171010. 136005.
 165091. 155347. 158511. 162216. 154015. 728575. 110419. 283470. 966382.
 170792. 216534. 961744. 159217. 270098. 211850. 214764. 231698. 248059.
 279235. 172188. 118368. 221858.     nan 220261. 976227. 280596. 101478.
 293118. 283931. 230572. 201531. 265254. 240573. 238961. 201966. 2

In [14]:
## Vamos remover esses valores faltantes da tabela filtrada

nas_de_interesse_2 = ['resultado_julgamento', 'filhos']
dados_limpos_3 = dados_filtrados_3.dropna(subset = nas_de_interesse_2)

print(len(dados_filtrados_3), '→', len(dados_limpos_3), 'observações após dropna nessas colunas')

## Isso significa que temos 136 observações com informação COMPLETA.

137 → 136 observações após dropna nessas colunas


In [15]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 3: resultado_julgamento vs existência de filhos

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'filhos', dados_limpos_3)
resultados['resultado_julgamento_vs_filhos'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs filhos ===
Tabela de contigência
filhos                0.0  1.0
resultado_julgamento          
0                       9   15
1                      98   14
Odds Ratio: 0.0857
P-value: 0.000001
Significância: *** (p < 0.001)

resultado_julgamento_vs_filhos:
  Odds Ratio: 0.0857
  P-valor: 0.000001
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Filhos**

1. Odds ratio = 0.0857. A chance de ser condenado quando há filhos é menor do que quando não há filhos.

2.  Dos casos com filho na amostra, 14 foram condenados e 15 absolvidos. 

3.  Dos casos sem filhos na amostra, 98 foram condenados e 9 absolvidos. 

**Conclusão:** *Ter filhos altera o resultado do julgamento*.

**• O p-valor está abaixo de 0.05 (0.000001), indicando uma associação forte entre resultado do julgamento e a existência de filhos.**

In [16]:
## Analisando consentimento familiar

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_4 = ['ID_processo','resultado_julgamento', 'consentimento_familiar']
dados_filtrados_4 = dados[variaveis_interesse_4].copy()
print(dados_filtrados_4)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_4)}")

     ID_processo  resultado_julgamento  consentimento_familiar
0       148088.0                     1                     0.0
1       203354.0                     1                     0.0
2       211280.0                     1                     0.0
3       264516.0                     1                     0.0
4       849912.0                     1                     0.0
..           ...                   ...                     ...
132     191972.0                     0                     1.0
133     240959.0                     0                     NaN
134     220480.0                     0                     1.0
135     221039.0                     0                     1.0
136     220755.0                     0                     1.0

[137 rows x 3 columns]
Total de observações originais: 137
Total de observações final: 137


In [17]:
## Verificando missing values

for var in variaveis_interesse_4:
    missing_count = dados_filtrados_4[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_4)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_4[var].unique()}")


## um dos missing values é o ID do processo. Vamos garantir que se elimine as observações com missing values apenas
## nas variáveis de interesse. 

ID_processo:
  Missing values: 1 (0.7%)
  Valores únicos: [148088. 203354. 211280. 264516. 849912. 804741. 254352. 202422. 224010.
 200221. 184262. 132092. 835236. 173028. 191800. 193481. 694900. 184789.
 174559. 143912. 280782. 215458. 885416. 913497. 208333. 212840. 871265.
 249385. 157443. 239068. 259005. 176898. 176979. 527774. 171531. 176559.
 169452. 149635. 175618. 140372. 143672. 155742. 158776. 231453. 134780.
 195164. 831541. 800685. 795482. 210952. 208602. 722014. 681080. 676155.
 197973. 197199. 208631. 237957. 147619. 183064. 201216. 169551. 208489.
 226153. 185259. 630372. 188501. 189990. 142704. 163831. 153688. 379763.
 477088. 382196. 387667. 160739. 185437. 163595. 176254. 171010. 136005.
 165091. 155347. 158511. 162216. 154015. 728575. 110419. 283470. 966382.
 170792. 216534. 961744. 159217. 270098. 211850. 214764. 231698. 248059.
 279235. 172188. 118368. 221858.     nan 220261. 976227. 280596. 101478.
 293118. 283931. 230572. 201531. 265254. 240573. 238961. 201966. 2

In [18]:
## Vamos remover esses valores faltantes da tabela filtrada

nas_de_interesse_3 = ['resultado_julgamento', 'consentimento_familiar']
dados_limpos_4 = dados_filtrados_4.dropna(subset = nas_de_interesse_3)

print(len(dados_filtrados_4), '→', len(dados_limpos_4), 'observações após dropna nessas colunas')

## Isso significa que temos 131 observações com informação COMPLETA.

137 → 131 observações após dropna nessas colunas


In [19]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 4: resultado_julgamento vs consentimento familiar

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'consentimento_familiar', dados_limpos_4)
resultados['resultado_julgamento_vs_consentimento_familiar'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs consentimento_familiar ===
Tabela de contigência
consentimento_familiar  0.0  1.0
resultado_julgamento            
0                         9   13
1                        84   25
Odds Ratio: 0.2060
P-value: 0.001450
Significância: ** (p < 0.01)

resultado_julgamento_vs_consentimento_familiar:
  Odds Ratio: 0.2060
  P-valor: 0.001450
  Significância: ** (p < 0.01)


## Interpretação

#### **Resultado do Julgamento vs Consentimento Familiar**

1. Odds ratio = 0.2060. A chance de ser condenado quando há consentimento familiar é menor do que quando não há consentimento familiar.

2.  Dos casos com consentimento familiar na amostra, 25 foram condenados e 13 absolvidos. 

3.  Dos casos sem consentimento familiar amostra, 84 foram condenados e 9 absolvidos. 

**Conclusão:** *O consentimento familiar altera o resultado do julgamento*.

**• O p-valor está abaixo de 0.05 (0.001450), indicando uma associação forte entre resultado do julgamento e o consentimento familiar.**

In [20]:
## Analisando consentimento da vítima

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_5 = ['ID_processo','resultado_julgamento', 'consentimento_vitima']
dados_filtrados_5 = dados[variaveis_interesse_5].copy()
print(dados_filtrados_5)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_5)}")


     ID_processo  resultado_julgamento  consentimento_vitima
0       148088.0                     1                     1
1       203354.0                     1                     1
2       211280.0                     1                     1
3       264516.0                     1                     1
4       849912.0                     1                     1
..           ...                   ...                   ...
132     191972.0                     0                     1
133     240959.0                     0                     1
134     220480.0                     0                     1
135     221039.0                     0                     1
136     220755.0                     0                     1

[137 rows x 3 columns]
Total de observações originais: 137
Total de observações final: 137


In [21]:
## Verificando missing values

for var in variaveis_interesse_5:
    missing_count = dados_filtrados_5[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_5)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_5[var].unique()}")

## Não há missing values, o total de observações com informação completa é 137. 

ID_processo:
  Missing values: 1 (0.7%)
  Valores únicos: [148088. 203354. 211280. 264516. 849912. 804741. 254352. 202422. 224010.
 200221. 184262. 132092. 835236. 173028. 191800. 193481. 694900. 184789.
 174559. 143912. 280782. 215458. 885416. 913497. 208333. 212840. 871265.
 249385. 157443. 239068. 259005. 176898. 176979. 527774. 171531. 176559.
 169452. 149635. 175618. 140372. 143672. 155742. 158776. 231453. 134780.
 195164. 831541. 800685. 795482. 210952. 208602. 722014. 681080. 676155.
 197973. 197199. 208631. 237957. 147619. 183064. 201216. 169551. 208489.
 226153. 185259. 630372. 188501. 189990. 142704. 163831. 153688. 379763.
 477088. 382196. 387667. 160739. 185437. 163595. 176254. 171010. 136005.
 165091. 155347. 158511. 162216. 154015. 728575. 110419. 283470. 966382.
 170792. 216534. 961744. 159217. 270098. 211850. 214764. 231698. 248059.
 279235. 172188. 118368. 221858.     nan 220261. 976227. 280596. 101478.
 293118. 283931. 230572. 201531. 265254. 240573. 238961. 201966. 2

In [22]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 5: resultado_julgamento vs consentimento da vítima

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'consentimento_vitima', dados_filtrados_5)
resultados['resultado_julgamento_vs_consentimento_vitima'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs consentimento_vitima ===
Tabela de contigência
consentimento_vitima   0   1
resultado_julgamento        
0                      0  24
1                     35  78
Odds Ratio: 0.0000
P-value: 0.000538
Significância: *** (p < 0.001)

resultado_julgamento_vs_consentimento_vitima:
  Odds Ratio: 0.0000
  P-valor: 0.000538
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Consentimento da Vítima**

1. Odds Ratio: 0 (tende a zero ou é indefinido). Há zero na tabela de contingência. Não existe nenhum caso de absolvição na amostra quando não há consentimento da vítima. 

2.  Dos casos com consentimento da vítima na amostra, 78 foram condenados e 24 absolvidos. 

3.  Dos casos sem consentimento da vítima na amostra, 35 foram condenados e 0 absolvidos. 

**Conclusão:** *O consentimento da vítima altera a decisão do julgamento. Mais especificamente, a chance de condenação
é muito alta - ou máxima - quando a vítima não consentiu. Quando há consentimento, a tomada de decisão é mista - depende do caso.*

**• O p-valor está abaixo de 0.05 (0.000538), indicando uma associação forte e estatisticamente significante entre
o resultado do julgamento e o consentimento da vítima. **